In [1]:
import climtas
import xarray
from glob import glob

In [2]:
import os
import dask.distributed

# Edit as desired
threads_per_worker = 1

try:
    c # Already running
except NameError:
    c = dask.distributed.Client(
        n_workers=int(os.environ.get('PBS_NCPUS', 1))//threads_per_worker,
        threads_per_worker=threads_per_worker,
        memory_limit=f'{4*threads_per_worker}gb',
        local_directory=os.path.join(os.environ.get('PBS_JOBFS'),'dask-worker-space')
    )
c

Client Scheduler: tcp://127.0.0.1:46133 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 2.00 GB


In [3]:
# Open the dataset

ds = xarray.open_mfdataset(sorted(glob('/g/data/ub4/era5/netcdf/surface/2T/*/2T_era5_global_*.nc')),
                           combine='nested',
                           concat_dim='time',
                           chunks={'latitude': 100, 'longitude': 100})
ds

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 359400)
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75
  * time       (time) datetime64[ns] 1979-01-01 ... 2019-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(744, 100, 100), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2019-03-13 14:42:07 GMT by grib_to_netcdf-2.10.0: /opt/ecmw...

In [4]:
# Convert to daily mean - or max, min etc.


def downsample(da, *, count):
    shape = list(da.shape)
    shape[0] //= count
    shape.insert(1, count)
    
    data = da.data.reshape(shape)
    
    dims = list(da.dims)
    dims.insert(1, 'sample')
    
    result = xarray.DataArray(data, dims=dims)
    
    result.coords[dims[0]] = da.coords[dims[0]][::count]
    for d in dims[2:]:
        result.coords[d] = da.coords[d]
    
    return result

t2m_daily = downsample(ds.t2m, count=24).mean('sample')
t2m_daily

<xarray.DataArray 'reshape-30a009221afac5831d30514f5f666b97' (time: 14975, latitude: 721, longitude: 1440)>
dask.array<mean_agg-aggregate, shape=(14975, 721, 1440), dtype=float32, chunksize=(31, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2019-12-31
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75

In [5]:
# Smooth out the data

t2m_smooth = t2m_daily.rolling(time=15, center=True).mean()
t2m_smooth

<xarray.DataArray (time: 14975, latitude: 721, longitude: 1440)>
dask.array<where, shape=(14975, 721, 1440), dtype=float32, chunksize=(31, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2019-12-31
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75

In [6]:
# Calculate percentiles
# The rolling mean will leave some NAN values at the start and end of the
# time series, these must be removed (e.g. by doing the rolling on a larger
# timeseries than you need and trimming the NANs off before the percentile)

t2m_percentile = climtas.apply_doy.percentile_doy(t2m_smooth.sel(time=slice('1980','2018')), 90)
t2m_percentile

<xarray.DataArray (latitude: 721, longitude: 1440, dayofyear: 366)>
dask.array<group_func, shape=(721, 1440, 366), dtype=float32, chunksize=(100, 100, 366), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366

In [ ]:
# Use the throttled saver to write to netcdf one chunk at a time

climtas.io.to_netcdf_throttled(t2m_percentile.to_dataset(name='t2m_percentile'),
                               '/g/data/w35/saw562/era5_heatwave_clim.nc')

In [ ]:
threshold = xarray.open_dataset('/g/data/w35/saw562/era5_heatwave_clim.nc',
                           chunks={'latitude': 200, 'longitude': 200}).t2m_percentile
threshold

In [ ]:
%matplotlib inline
threshold.sel(latitude=-37.8, longitude=144.9, method='nearest').plot()

In [ ]:
t2m_daily.groupby('time.dayofyear') > threshold

In [ ]:
climtas.event.find_events(t2m_daily.groupby('time.dayofyear') > threshold, min_duration=3)

In [ ]:
t2m_daily.groupby('time.dayofyear') > threshold.transpose('dayofyear','latitude','longitude')